<a href="https://colab.research.google.com/github/gracee331/graczzz/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.9 MB/s eta 0:00:00


In [2]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent,TextMessageContent
from linebot.v3.messaging import (
    Configuration,ApiClient,MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage

)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route('/callback', methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)  # Line平台傳訊息的內容

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_genai(question):
    response = model.generate_content(question)
    return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
            # 回覆貼圖
            reply = StickerMessage(
               package_id='446',
               sticker_id='1989'
            )
        elif action == 'image':
            # 回覆圖片
            reply = ImageMessage(
               original_content_url='https://cdn.pixabay.com/photo/2014/03/14/20/13/dog-287420_1280.jpg',
               preview_image_url='https://cdn.pixabay.com/photo/2014/03/14/20/13/dog-287420_1280.jpg'
            )
        elif action == 'video':
            reply = VideoMessage(
              original_content_url='https://cdn.pixabay.com/video/2016/07/26/4006-176282263_large.mp4',
              preview_image_url='https://cdn.pixabay.com/photo/2016/11/29/04/19/ocean-1867285_1280.jpg'
            )
        elif action == 'location':
            reply = LocationMessage(
              title="中山大學",
              address="804高雄市鼓山區蓮海路70號",
              latitude=22.628655579154614,
              longitude=120.26482091633952
            )

        else:
            response = ask_genai(action)
            reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() # for colab
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8872-34-168-89-110.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
